In [0]:
# first we uninstall tf version 1.4 

!pip uninstall tensorflow 

In [0]:
#importing...
# installing tf 2.0 and augmentor

!pip install -q tensorflow-gpu==2.0.0-beta1
!pip install Augmentor
import Augmentor
from shutil import copy
import shutil
import os
from tqdm import tqdm
import cv2
import numpy as np
from random import shuffle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
#mport tensorflow_datasets as tfds
tf.__version__


In [0]:
#copy all training images to a new folder named new_data


shutil.rmtree("New_data", ignore_errors = True)
os.mkdir("New_data")
dest = "New_data"
src = ['category 1', 'category 2', 'category 3', 'category 4']

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# copy images to one folder
#do note that due ro for loop it will print 100% four times
for i in src:
    f = os.path.join('/content/drive/My Drive/Image_2.zip (Unzipped Files)/Image_2/Input/Dataset/train/',i)
    for img in tqdm(os.listdir(f)): 
            print(img)
            path = os.path.join(f,img)
            print(path)
            copy(path, dest)

In [0]:
# path to new data directory
data_dir = "New_data"
# image size that we want
img_size = 150

#path to augmented images
output = "New_data/output"

In [0]:
#producing augmented images

p = Augmentor.Pipeline(data_dir)
p.rotate270(probability=0.2)
p.flip_top_bottom(0.2)
p.flip_left_right(probability=0.2)
p.random_distortion(probability=0.1, grid_width=4, grid_height=4, magnitude=8)
#p.zoom_random(0.1, percentage_area=0.8)
p.sample(10000) 

In [0]:
#producing labels


def label_img(img):
    label = img.split('_')[3]
    if label == 'category 1': return 0
    elif label == 'category 2': return 1
    elif label == 'category 3': return 2
    elif label == 'category 4': return 3

In [0]:
#creating numpyfile from images in output folder
#this function can create both training and test data
def tdata(flag):
    output = "New_data/output"
    if flag == 'test':
        output = "/content/drive/My Drive/Image_2.zip (Unzipped Files)/Image_2/Input/Dataset/test"
    data = []
    ids = {}
    values = []
    i = 0
    for img in tqdm(os.listdir(output)):
          
        path = os.path.join(output,img)
        if flag == 'train':
            
            label = label_img(img)
            
        
        
        try:
          
            
            im = cv2.resize(cv2.imread(path),(img_size, img_size))
            grayImage = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            normalizedImg = np.zeros((150, 150, 1))
            normalizedImg = cv2.normalize(grayImage,  normalizedImg, 0, 255, cv2.NORM_MINMAX)
            if flag == 'train':
                data.append([np.array(normalizedImg),  np.array(label)])
                
            #for test data we will include image name , so that we can keep track of the prediction for images
            
            else:
                values.append(img) 
                ids[i] = values[i]
                data.append([np.array(normalizedImg)])
                i +=1 

        except Exception as e:
                    print('skipping...')
                    print(e)
        
                   
   
    if flag == 'train':
            # we shuffle in case of training but not in test data
            shuffle(data)
            shutil.rmtree('train_data.npy', ignore_errors = True)
            np.save('train_data.npy', data)
    else :
        shutil.rmtree('test_data.npy', ignore_errors = True)
        np.save('test_data.npy', data)
        
        return ids  

In [0]:
tdata('train')

In [0]:
#opening numpyfile

data = np.load('train_data.npy',allow_pickle=True)
print(data.shape)

In [0]:
#spliting into train and eval set
np.random.shuffle(data)
training, evaluate = data[:8000,:], data[8000:,:]

In [0]:
print(training.shape)
print(training[0,1])

In [0]:
#ploting first image

for i in range(5):
        print(training[i,1])
        plt.imshow(training[i,0], cmap="gray")
        plt.show()

In [0]:

#spliting features and labels

FEATURES = []
LABELS = []

for feature, label in  training:
    FEATURES.append(feature)
    LABELS.append(label)
        

In [0]:
X_TRAIN = np.array(FEATURES).reshape(-1, img_size, img_size, 1)
Y_TRAIN = np.array(LABELS)
print(X_TRAIN.shape,Y_TRAIN.shape)

In [0]:
X_EVAL = []
Y_EVAL = []

for feature, label in  evaluate:
    X_EVAL.append(feature)
    Y_EVAL.append(label)
    
X_EVAL = np.array(X_EVAL).reshape(-1, img_size, img_size, 1)
Y_EVAL = np.array(Y_EVAL)

buffer = 60000
BATCH_SIZE = 100

# making dataset

train_dataset = tf.data.Dataset.from_tensor_slices((X_TRAIN,Y_TRAIN)).shuffle(buffer).batch(BATCH_SIZE)
eval_dataset = tf.data.Dataset.from_tensor_slices((X_EVAL,Y_EVAL)).shuffle(buffer).batch(BATCH_SIZE)



In [0]:
# CNN model

model = tf.keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001), input_shape=[150, 150,1]))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))

model.add(layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.3))

# model.add(layers.Conv2D(128, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(layers.Dense(4, activation='softmax'))   
model.summary()

In [0]:

#checkpoints
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_weights_only=True,verbose=1)

#tensorboard 
callbacks = tf.keras.callbacks.TensorBoard('./logs_keras')

#earlystopping

#callback2 = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)


In [0]:

#compile and running training

model.compile(optimizer=tf.compat.v2.optimizers.Adadelta(0.01),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.fit(train_dataset, epochs=200, verbose=1, validation_data=(eval_dataset), callbacks=[callbacks, cp_callback])


In [0]:
#train from latest check point
tf.train.latest_checkpoint(checkpoint_dir)


In [0]:
!tensorboard --logdir ./logs_keras

In [0]:
#create test data in numpy file

idds = tdata('test')

In [0]:
idds


In [0]:
# load test data and seperate test images and their name

data = np.load('test_data.npy',allow_pickle=True)
        
d = data.squeeze().reshape(-1, img_size, img_size, 1)


#d = data[0].squeeze().reshape(-1, 150, 150, 1)
print(d.shape)

In [0]:
#getting prediction

prid = model.predict(d)
print(prid)
for i in prid:
      print(np.argmax(i))

In [0]:

import pandas as pd
dictn = {0:'category 1', 1:'category 2', 2 :'category 3', 3:'category 4'}

In [0]:
#writting result to a csv file using pandas


prid_data = pd.DataFrame({'image_Id':idds,'predicted_labels':dictn[np.argmax(prid)]})
#age_Id'] = data['image_Id'].astype('int')
#prid_data['predicted_labels'] = data['predicted_labels'].astype('str')
prid_data.to_csv('submission.csv')
print(prid_data)